# Price Integration

## 1. Introduction 

We have two databases. The first is our internal product database, which does not contain any pricing information. The second is an external database containing numerous products with associated prices. Our objective is to integrate these prices into our product database. To do this, we must compare products from both databases and perform a "match" based on product names.

To achieve this, I used the RapidFuzz library to compare product names using a similarity score. However, a problem arose: the two databases were not in the same language. One contained products in English, while the other (the external database) contained products in various languages, but primarily in French.

This creates a significant issue. For example, the word "Fraise" in French translates to "Strawberry" in English. These words have no lexical resemblance for a standard matching algorithm. I initially attempted to translate one of the files into English using the Google Translate library, but this proved unfeasible due to the volume of data (over 100,000 entries), with an estimated processing time of 24 hours.

Consequently, Kali provided a better-organized external price file to reduce processing time. Meanwhile, Antoine provided a file containing the top 1,000 best-selling products from our database (representing 90% of sales). This strategy significantly reduced the volume of values requiring translation.

Input files:

- top_1000_products.csv: Top 1,000 best-selling products from our database.

- prices_final_imputed_translated.csv: External database containing prices.

Output file:

- final/product_with_price.csv: Top 1,000 best-selling products from our database with integrated prices.

## 2. Preprocessing

### 2.1 Conversion of CSV files to Excel for visual inspection.

Input: "top_1000_products.csv" and "Translate_PriceDB/Translate_PriceDB/prices_final_imputed_translated.csv"  
Output: "a_traiter/product_base.xlsx" and "a_traiter/product_prices_non_traite.xlsx"  

In [2]:
import pandas as pd
import os

# --- CONFIGURATION DES CHEMINS ---
# Chemins d'entrée (tels que tu me les as donnés)
FILE_1_CSV = "top_1000_products.csv"
FILE_2_CSV = "prices_final_imputed_translated.csv"

# Dossier et fichiers de sortie
OUT_DIR = "a_traiter"
FILE_1_XLSX = os.path.join(OUT_DIR, "product_base.xlsx")
FILE_2_XLSX = os.path.join(OUT_DIR, "product_prices_non_traite.xlsx")

def convert_to_excel():
    print("--- Démarrage de la conversion CSV -> Excel ---")

    # 1. Création du dossier de sortie s'il n'existe pas
    if not os.path.exists(OUT_DIR):
        print(f"Création du dossier '{OUT_DIR}'...")
        os.makedirs(OUT_DIR)

    # --- TRAITEMENT FICHIER 1 : PRODUITS ---
    print(f"\n1. Traitement de {FILE_1_CSV}...")
    if os.path.exists(FILE_1_CSV):
        try:
            # Séparateur virgule "," pour ce fichier
            df_prod = pd.read_csv(FILE_1_CSV, sep=',')
            
            print(f"   -> {len(df_prod)} lignes lues.")
            print(f"   -> Écriture vers {FILE_1_XLSX}...")
            
            df_prod.to_excel(FILE_1_XLSX, index=False, engine='openpyxl')
            print("   -> Succès.")
        except Exception as e:
            print(f"   -> ERREUR : {e}")
    else:
        print(f"   -> ERREUR : Le fichier {FILE_1_CSV} est introuvable.")

    # --- TRAITEMENT FICHIER 2 : PRIX ---
    print(f"\n2. Traitement de {FILE_2_CSV}...")
    if os.path.exists(FILE_2_CSV):
        try:
            # Séparateur point-virgule ";" d'après ton extrait
            # On utilise engine='python' pour plus de tolérance sur les erreurs de parsing
            df_price = pd.read_csv(FILE_2_CSV, sep=';', engine='python')
            
            print(f"   -> {len(df_price)} lignes lues.")
            
            # Nettoyage léger pré-Excel :
            # Ton extrait montre une première colonne vide (le CSV commence par ;)
            # Si une colonne est entièrement vide et sans nom (Unnamed), on peut l'enlever pour la lisibilité
            df_price = df_price.dropna(how='all', axis=1) 
            
            print(f"   -> Écriture vers {FILE_2_XLSX}...")
            # Utilisation de xlsxwriter pour gérer les caractères spéciaux potentiels
            df_price.to_excel(FILE_2_XLSX, index=False, engine='xlsxwriter')
            print("   -> Succès.")
        except Exception as e:
            print(f"   -> ERREUR : {e}")
    else:
        print(f"   -> ERREUR : Le fichier {FILE_2_CSV} est introuvable.")
        print("      Vérifie que le chemin (Translate_PriceDB/...) est correct.")

    print("\n--- Opération terminée ---")

if __name__ == "__main__":
    convert_to_excel()

--- Démarrage de la conversion CSV -> Excel ---

1. Traitement de top_1000_products.csv...
   -> 1000 lignes lues.
   -> Écriture vers a_traiter\product_base.xlsx...
   -> Succès.

2. Traitement de prices_final_imputed_translated.csv...
   -> 181734 lignes lues.
   -> Écriture vers a_traiter\product_prices_non_traite.xlsx...
   -> Succès.

--- Opération terminée ---


### 2.2 Removal of Unnecessary Columns for Price Integration

Columns to retain from the product_prices_non_traite file (the final file will contain only the following columns):  

- product_name: Join key. Shared attribute with the product_base file that will enable linking a price to a product.  

- price: The displayed retail price value.  

- price_without_discount: The "True Price" (excluding promotions) to avoid skewing revenue simulations.  

- currency: The currency (e.g., $1.00 ≠ €1.00).  

- category_tag: Product category.  

Input: a_traiter/product_prices_non_traite  
Output: a_traiter/product_prices_traite  

In [ ]:
# --- CONFIGURATION ---
INPUT_FILE = 'a_traiter/product_prices_non_traite.xlsx'
OUTPUT_FILE = 'a_traiter/product_prices_traite.xlsx'

# Liste stricte des colonnes demandées
COLS_TO_KEEP = [
    'product_name', 
    'price', 
    'price_without_discount', 
    'currency', 
    'category_tag'
]

def filter_and_clean():
    print("--- Démarrage du filtrage et nettoyage des prix ---")

    if not os.path.exists(INPUT_FILE):
        print(f"ERREUR : Le fichier {INPUT_FILE} est introuvable.")
        return

    # 1. Chargement du fichier
    print(f"Lecture de {INPUT_FILE}...")
    try:
        df = pd.read_excel(INPUT_FILE, engine='openpyxl')
    except Exception as e:
        print(f"Erreur de lecture : {e}")
        return

    # 2. Vérification de la présence des colonnes
    missing_cols = [col for col in COLS_TO_KEEP if col not in df.columns]
    if missing_cols:
        print(f"ATTENTION : Colonnes manquantes dans le fichier source : {missing_cols}")
        return

    # 3. Sélection des colonnes
    print("Conservation uniquement des 5 colonnes cibles...")
    df_clean = df[COLS_TO_KEEP].copy()

    # 4. Logique Business (Consolidation du Prix)
    # On crée 'final_price' ici pour que le fichier 'traité' soit prêt à l'emploi.
    # Règle : Si 'price_without_discount' existe, on le prend. Sinon on prend 'price'.
    print("Calcul du 'final_price' (Priorité au prix hors remise)...")
    df_clean['final_price'] = df_clean['price_without_discount'].fillna(df_clean['price'])

    # 5. Nettoyage des lignes invalides
    # On supprime si le Nom est vide OU si le Prix Final est vide
    initial_len = len(df_clean)
    df_clean = df_clean.dropna(subset=['product_name', 'final_price'])
    cleaned_len = len(df_clean)
    
    print(f"Nettoyage : {initial_len - cleaned_len} lignes vides supprimées.")

    # 6. Sauvegarde
    print(f"Sauvegarde vers {OUTPUT_FILE}...")
    # On garde les colonnes sources + le prix final calculé
    cols_export = COLS_TO_KEEP + ['final_price']
    
    # Réorganisation pour avoir final_price juste après product_name pour la lisibilité
    cols_ordered = ['product_name', 'final_price', 'currency', 'category_tag', 'price', 'price_without_discount']
    
    # On s'assure de ne prendre que ce qui existe (au cas où category_tag serait vide/absent)
    final_cols = [c for c in cols_ordered if c in df_clean.columns]
    
    df_clean[final_cols].to_excel(OUTPUT_FILE, index=False)
    print("--- Succès ! Fichier généré. ---")

if __name__ == "__main__":
    filter_and_clean()

--- Démarrage du filtrage et nettoyage des prix ---
Lecture de a_traiter/product_prices_non_traite.xlsx...
Conservation uniquement des 5 colonnes cibles...
Calcul du 'final_price' (Priorité au prix hors remise)...
Nettoyage : 73810 lignes vides supprimées.
Sauvegarde vers a_traiter/product_prices_traite.xlsx...
--- Succès ! Fichier généré. ---


### 2.3 Processing the 'category_tag' Column

We will now perform processing on the file 'a_traiter/product_prices_traite.xlsx'.

'category_tag' column: For each row where the value is not null, the data follows the format "en:value" or "fr:value". We must retain only the value (everything to the right of the ":"). The language prefix is unnecessary; the product data must be "clean".

The Excel file is then saved using the same filename (overwriting the original).

In [ ]:
# --- CONFIGURATION ---
FILE_PATH = 'a_traiter/product_prices_traite.xlsx'

def clean_tags():
    print(f"--- Nettoyage de la colonne 'category_tag' dans {FILE_PATH} ---")

    # 1. Vérification de l'existence du fichier
    if not os.path.exists(FILE_PATH):
        print(f"ERREUR : Le fichier {FILE_PATH} est introuvable.")
        return

    # 2. Chargement du fichier Excel
    print("Chargement du fichier...")
    try:
        df = pd.read_excel(FILE_PATH, engine='openpyxl')
    except Exception as e:
        print(f"Erreur lors de la lecture : {e}")
        return

    # 3. Fonction de nettoyage
    def remove_prefix(text):
        """
        Si le texte est 'en:bananas', retourne 'bananas'.
        Si le texte est vide ou sans ':', retourne le texte original.
        """
        if isinstance(text, str) and ':' in text:
            # On coupe au PREMIER ':' rencontré et on prend la suite
            # split(':', 1) permet de gérer les cas où la valeur contiendrait aussi un ':'
            return text.split(':', 1)[1]
        return text

    # 4. Application du nettoyage
    if 'category_tag' in df.columns:
        print("Nettoyage des préfixes (en:, fr:, etc.)...")
        # On applique la fonction sur toute la colonne
        df['category_tag'] = df['category_tag'].apply(remove_prefix)
    else:
        print("ATTENTION : La colonne 'category_tag' n'existe pas dans ce fichier.")

    # 5. Sauvegarde (Écrasement du fichier)
    print("Sauvegarde (écrasement du fichier original)...")
    try:
        df.to_excel(FILE_PATH, index=False, engine='openpyxl')
        print("SUCCÈS : Le fichier a été mis à jour.")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde : {e}")

if __name__ == "__main__":
    clean_tags()

--- Nettoyage de la colonne 'category_tag' dans a_traiter/product_prices_traite.xlsx ---
Chargement du fichier...
Nettoyage des préfixes (en:, fr:, etc.)...
Sauvegarde (écrasement du fichier original)...
SUCCÈS : Le fichier a été mis à jour.


## 3. Translation

We will translate the product_name column from the file "matching/product_base.xlsx" into a new file: "matching/base_product_fr.xlsx".  

Translation is performed from English to French into a new column, product_fr (the original English product_name column must be retained).  

Consequently, our database file will ultimately contain two product name columns: one in English and one in French. For a more robust comparison, we will subsequently evaluate both columns during the matching process and select the highest score between the English and French versions.  

We use the tqdm library to monitor progress and the GoogleTranslator library for the translation.  

In [ ]:
import pandas as pd
import os
from deep_translator import GoogleTranslator
from tqdm import tqdm

# --- CONFIGURATION ---
INPUT_FILE = 'a_traiter/product_base.xlsx'
OUTPUT_FILE = 'a_traiter/product_base_fr.xlsx'

def translate_base():
    print("--- Démarrage de la traduction (EN -> FR) ---")

    # 1. Vérification du fichier
    if not os.path.exists(INPUT_FILE):
        print(f"ERREUR : Le fichier {INPUT_FILE} est introuvable.")
        return

    # 2. Chargement
    print(f"Lecture de {INPUT_FILE}...")
    try:
        df = pd.read_excel(INPUT_FILE, engine='openpyxl')
    except Exception as e:
        print(f"Erreur de lecture : {e}")
        return

    print(f"   -> {len(df)} produits à traduire.")

    # 3. Initialisation du traducteur
    # source='en' (Anglais) -> target='fr' (Français)
    translator = GoogleTranslator(source='en', target='fr')

    # 4. Fonction de traduction sécurisée
    def safe_translate(text):
        if not isinstance(text, str) or len(text) < 2:
            return text
        try:
            return translator.translate(text)
        except Exception:
            return text  # En cas d'erreur (timeout), on retourne le texte original

    # 5. Application avec barre de progression (tqdm)
    print("Traduction en cours (veuillez patienter)...")
    
    # On active tqdm pour pandas
    tqdm.pandas(desc="Progression")
    
    # Création de la nouvelle colonne 'product_name_fr'
    # On garde 'product_name' intact
    df['product_name_fr'] = df['product_name'].progress_apply(safe_translate)

    # 6. Sauvegarde
    print(f"Sauvegarde vers {OUTPUT_FILE}...")
    
    # On réorganise les colonnes pour avoir la traduction juste après le nom original
    cols = list(df.columns)
    if 'product_name' in cols and 'product_name_fr' in cols:
        # On place product_name_fr juste après product_name
        index_original = cols.index('product_name')
        cols.remove('product_name_fr')
        cols.insert(index_original + 1, 'product_name_fr')
        df = df[cols]

    df.to_excel(OUTPUT_FILE, index=False, engine='openpyxl')
    print("--- Terminé avec succès ! ---")

if __name__ == "__main__":
    translate_base()

--- Démarrage de la traduction (EN -> FR) ---
Lecture de a_traiter/product_base.xlsx...
   -> 1000 produits à traduire.
Traduction en cours (veuillez patienter)...


Progression: 100%|██████████| 1000/1000 [18:12<00:00,  1.09s/it]

Sauvegarde vers a_traiter/product_base_fr.xlsx...
--- Terminé avec succès ! ---


## 4. Normalization

We will normalize the product_name column values in both files to facilitate the comparison process.

We will remove uppercase letters, accents, and leading/trailing whitespace. This preprocessing step is designed to optimize the matching algorithm.

Once completed, the two files are saved into the new "matching/" directory.

Note: The final output must contain the original, correct product name (un-normalized). The standardization performed on these two files is strictly for matching purposes.

In [10]:
import pandas as pd
import os
import re
from unidecode import unidecode

# --- CONFIGURATION ---
INPUT_DIR = 'a_traiter'
OUTPUT_DIR = 'matching'

FILES_TO_PROCESS = [
    'product_prices_traite.xlsx',
    'product_base_fr.xlsx'
]

def normalize_text(text):
    """
    Fonction de nettoyage strict pour le matching :
    - Minuscules
    - Suppression des accents (café -> cafe)
    - Remplacement de la ponctuation par des espaces
    - Suppression des espaces multiples et trim
    """
    if not isinstance(text, str):
        return ""
    
    # 1. Minuscules
    text = text.lower()
    
    # 2. Retrait des accents (nécessite unidecode)
    text = unidecode(text)
    
    # 3. On ne garde que les lettres (a-z) et les chiffres (0-9)
    text = re.sub(r'[^a-z0-9]', ' ', text)
    
    # 4. On nettoie les espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def prepare_files():
    print("--- Préparation des fichiers pour le Matching ---")
    
    # Création du dossier de sortie
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    for filename in FILES_TO_PROCESS:
        input_path = os.path.join(INPUT_DIR, filename)
        output_path = os.path.join(OUTPUT_DIR, filename)
        
        if not os.path.exists(input_path):
            print(f"ERREUR : Fichier introuvable -> {input_path}")
            continue

        print(f"Traitement de {filename}...")
        
        try:
            # Lecture
            df = pd.read_excel(input_path, engine='openpyxl')
            
            # --- LOGIQUE SPÉCIFIQUE SELON LE FICHIER ---
            
            if 'product_base_fr.xlsx' in filename:
                print("   -> Mode : Double Normalisation (Original & FR)")
                
                # 1. Normalisation de la colonne originale (product_name -> product_name_normalized)
                if 'product_name' in df.columns:
                    df['product_name_normalized'] = df['product_name'].apply(normalize_text)
                else:
                    print("      ATTENTION: 'product_name' introuvable.")

                # 2. Normalisation de la colonne traduite (product_name_fr -> product_name_fr_normalized)
                if 'product_name_fr' in df.columns:
                    df['product_name_fr_normalized'] = df['product_name_fr'].apply(normalize_text)
                else:
                    print("      ATTENTION: 'product_name_fr' introuvable.")
                    
            else:
                # Cas du fichier de PRIX (product_prices_traite.xlsx)
                print("   -> Mode : Normalisation Simple")
                if 'product_name' in df.columns:
                    df['product_name_normalized'] = df['product_name'].apply(normalize_text)
                else:
                    print("      ERREUR : La colonne 'product_name' est absente.")
                    continue

            # Sauvegarde
            print(f"   -> Sauvegarde dans {OUTPUT_DIR}/...")
            df.to_excel(output_path, index=False, engine='openpyxl')
            
        except Exception as e:
            print(f"   -> ERREUR CRITIQUE : {e}")

    print("-" * 30)
    print("Succès ! Les fichiers sont prêts dans le dossier 'matching/'.")

if __name__ == "__main__":
    prepare_files()

--- Préparation des fichiers pour le Matching ---
Traitement de product_prices_traite.xlsx...
   -> Mode : Normalisation Simple
   -> Sauvegarde dans matching/...
Traitement de product_base_fr.xlsx...
   -> Mode : Double Normalisation (Original & FR)
   -> Sauvegarde dans matching/...
------------------------------
Succès ! Les fichiers sont prêts dans le dossier 'matching/'.


## 5. Matching

We perform the matching process by comparing data from the "matching/product_base_fr.xlsx" file against the "matching/product_price_traite.xlsx" file.

For each row in product_base_fr, we compare the following two columns: product_name_normalized and product_name_fr_normalized against the product_name_normalized column of the product_price_traite file. We then retrieve the highest match score relative to the values in the price file.

Here, we compare the product in both English and French to retain the higher of the two scores, given that the language of the products in the product_price_traite file is not known in advance. This ensures greater robustness and superior results.

Once the highest score is identified, we retrieve the currency and price values to append them to the product_base_fr file.

The final file will contain the following columns:
rank, product_id, product_name, product_name_fr, nb_purchases, currency, price, score, source

The source column will contain one of two values: "fr" or "en". If the highest score was derived from the product_name_normalized value, we assign "en"; otherwise, if the highest score was derived from the product_name_fr_normalized value, we assign "fr".

Output: "final/product_with_price.xlsx"

We also export it to CSV: "final/product_with_price.csv"

In [11]:
import pandas as pd
import os
from rapidfuzz import process, fuzz
from tqdm import tqdm

# --- CONFIGURATION ---
INPUT_BASE = 'matching/product_base_fr.xlsx'
# Attention : je corrige le nom ici ('prices' au pluriel comme généré précédemment)
INPUT_PRICES = 'matching/product_prices_traite.xlsx' 

OUTPUT_DIR = 'final'
OUTPUT_XLSX = os.path.join(OUTPUT_DIR, 'product_with_price.xlsx')
OUTPUT_CSV = os.path.join(OUTPUT_DIR, 'product_with_price.csv')

def run_matching():
    print("--- Démarrage du Matching Final (Duel FR vs EN) ---")

    # 1. Chargement des fichiers
    print("Chargement des données...")
    if not os.path.exists(INPUT_BASE) or not os.path.exists(INPUT_PRICES):
        print("ERREUR : Fichiers d'entrée introuvables dans le dossier 'matching/'.")
        return

    df_base = pd.read_excel(INPUT_BASE, engine='openpyxl')
    df_prices = pd.read_excel(INPUT_PRICES, engine='openpyxl')

    print(f"   -> Base produits (Instacart) : {len(df_base)} lignes")
    print(f"   -> Base prix (OpenFoodFacts) : {len(df_prices)} lignes")

    # 2. Préparation pour RapidFuzz
    # On crée une liste des noms normalisés du fichier PRIX pour la recherche
    # On s'assure que ce sont des chaines de caractères (str)
    price_choices = df_prices['product_name_normalized'].astype(str).tolist()

    # On prépare les listes pour stocker les résultats
    results_currency = []
    results_price = []
    results_score = []
    results_source = []

    print("Calcul des scores en cours...")
    
    # On itère sur chaque produit de la base
    # tqdm affiche une barre de progression
    for i, row in tqdm(df_base.iterrows(), total=df_base.shape[0]):
        
        # --- MATCHING ANGLAIS ---
        name_en = str(row['product_name_normalized'])
        # process.extractOne retourne : (match_string, score, index)
        match_en = process.extractOne(name_en, price_choices, scorer=fuzz.token_sort_ratio)
        
        # --- MATCHING FRANCAIS ---
        name_fr = str(row['product_name_fr_normalized'])
        match_fr = process.extractOne(name_fr, price_choices, scorer=fuzz.token_sort_ratio)

        # --- LE DUEL : QUI A LE MEILLEUR SCORE ? ---
        # match_en[1] est le score, match_en[2] est l'index dans df_prices
        
        score_en = match_en[1]
        score_fr = match_fr[1]

        if score_en >= score_fr:
            # L'anglais gagne (ou égalité)
            best_score = score_en
            best_index = match_en[2]
            source = "en"
        else:
            # Le français gagne
            best_score = score_fr
            best_index = match_fr[2]
            source = "fr"

        # Récupération des données du gagnant dans le fichier PRIX
        price_row = df_prices.iloc[best_index]
        
        results_score.append(best_score)
        results_source.append(source)
        results_price.append(price_row['final_price'])
        results_currency.append(price_row['currency'])

    # 3. Construction du DataFrame final
    print("Construction du fichier final...")
    df_base['currency'] = results_currency
    df_base['price'] = results_price
    df_base['score'] = results_score
    df_base['source'] = results_source

    # Sélection des colonnes demandées
    final_cols = [
        'rank', 
        'product_id', 
        'product_name', 
        'product_name_fr', 
        'nb_purchases', 
        'currency', 
        'price', 
        'score', 
        'source'
    ]
    
    # On filtre pour ne garder que ces colonnes (si elles existent)
    cols_to_export = [c for c in final_cols if c in df_base.columns]
    df_final = df_base[cols_to_export]

    # 4. Sauvegarde
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    print(f"Sauvegarde Excel : {OUTPUT_XLSX}")
    df_final.to_excel(OUTPUT_XLSX, index=False, engine='openpyxl')
    
    print(f"Sauvegarde CSV : {OUTPUT_CSV}")
    df_final.to_csv(OUTPUT_CSV, index=False)

    print("-" * 30)
    print("TERMINÉ ! Tu peux analyser les scores dans le dossier 'final/'.")

if __name__ == "__main__":
    run_matching()

--- Démarrage du Matching Final (Duel FR vs EN) ---
Chargement des données...
   -> Base produits (Instacart) : 1000 lignes
   -> Base prix (OpenFoodFacts) : 107924 lignes
Calcul des scores en cours...


100%|██████████| 1000/1000 [02:53<00:00,  5.78it/s]


Construction du fichier final...
Sauvegarde Excel : final\product_with_price.xlsx
Sauvegarde CSV : final\product_with_price.csv
------------------------------
TERMINÉ ! Tu peux analyser les scores dans le dossier 'final/'.


## 6. Conclusion

I selected the RapidFuzz library for its efficiency in string similarity comparison. Additionally, resource constraints prevented the use of an AI model capable of native cross-lingual product matching (such as ChatGPT). Furthermore, this technology delivers robust results through its scoring mechanism.  

Scoring Mechanism:  
We utilize the RapidFuzz algorithm with the token_sort_ratio method. This approach disregards word order (e.g., 'Organic Milk' = 'Milk Organic') and tolerates minor spelling variations.  

For each product, we perform two parallel searches within the price database: one using the English name and one using the French translation. We systematically retain the result with the highest confidence score (the Max), allowing us to retrieve the most relevant price regardless of the product's source language.  